# BloodborneIF: Text based RPG with LLM parsing

### Game State Controller

- Player weapon upgrade level
- Current player health
- Current Player insight
- Current player location
- Objects that can be interacted with in the current location
- Player inventory

In [ ]:
from collections import defaultdict
import io
import sys
import os

print_to_stdout = True
print_to_devnull = False

if print_to_stdout:
    game_io = sys.stdout
elif print_to_devnull:
    game_io = open(os.devnull, 'w')
else:
    game_io = io.StringIO()

def print_game_output(*args, **kwargs):
    """Helper used by the game engine to print text to the player."""
    kwargs['file'] = game_io
    print(*args, **kwargs)


In [ ]:

# --- Metadata loading from Data.zip for items and characters ---

DATA_ZIP_PATH = "Data.zip"

def _load_metadata_from_data_zip(zip_path: str = DATA_ZIP_PATH):
    """Load item and character metadata from Data.zip, if it exists.

    Returns:
        item_meta_by_name: dict[str, dict]  # key: lowercased item name
        char_meta_by_id: dict[str, dict]   # key: character id
    """
    item_meta_by_name = {}
    char_meta_by_id = {}

    if not os.path.isfile(zip_path):
        return item_meta_by_name, char_meta_by_id

    with zipfile.ZipFile(zip_path, "r") as z:
        for name in z.namelist():
            if name.startswith("Data/Items/") and name.endswith(".json"):
                with z.open(name) as f:
                    data = json.load(f)
                item_name = data.get("name", data.get("id", "")).lower()
                item_meta_by_name[item_name] = data

        for name in z.namelist():
            if name.startswith("Data/Characters/") and name.endswith(".json"):
                with z.open(name) as f:
                    data = json.load(f)
                char_id = data.get("id")
                if char_id:
                    char_meta_by_id[char_id] = data

    return item_meta_by_name, char_meta_by_id

try:
    ITEM_META, CHARACTER_META = _load_metadata_from_data_zip()
except Exception:
    ITEM_META, CHARACTER_META = {}, {}


## Core World Objects

We keep the same structure as the Action Castle engine:
- **Location**: a node in the world graph
- **Item**: an object that can be in a location or in the player's inventory


In [11]:

class Location:
    """Represents a place in the world.

    Tracks:
      - id, name, description
      - insight_level (how eldritch the area feels)
      - connections to other locations (directions)
      - items present (stacked via item_counts)
      - characters/NPCs present (id -> count)
      - whether the player has ever visited before
    """

    def __init__(self, name, description, insight_level: int = 0, loc_id: str = None):
        self.id = loc_id if loc_id is not None else name
        self.name = name
        self.description = description

        self.properties = defaultdict(bool)
        self.connections = {}           # direction -> Location
        self.travel_descriptions = {}   # direction -> string narrative

        # Items & counts
        self.items = {}                 # item_name -> Item
        self.item_counts = defaultdict(int)  # item_name -> quantity

        # Characters & counts
        self.characters = defaultdict(int)   # character_id -> count

        # Puzzles / gates per direction
        self.blocks = {}

        # First-visit tracking
        self.has_been_visited = False

        # How "cosmic" / eldritch this location feels
        self.insight_level = insight_level

    # ----- Generic properties -----
    def set_property(self, property_name, property_bool=True):
        self.properties[property_name] = property_bool

    def get_property(self, property_name):
        return self.properties[property_name]

    # ----- Connections & blocks -----
    def add_connection(self, direction, connected_location, travel_description=""):
        direction = direction.lower()
        self.connections[direction] = connected_location
        self.travel_descriptions[direction] = travel_description

        # Automatically add simple reverse connections
        reverse = {
            "north": "south", "south": "north",
            "east": "west",   "west": "east",
            "up": "down",     "down": "up",
        }
        if direction in reverse:
            rev = reverse[direction]
            if rev not in connected_location.connections:
                connected_location.connections[rev] = self
                connected_location.travel_descriptions[rev] = ""

    def add_block(self, blocked_direction, block_description, preconditions):
        """Add a puzzle/block to a direction.
        `preconditions` is a list of callables: game -> bool.
        """
        self.blocks[blocked_direction] = (block_description, preconditions)

    def is_blocked(self, direction, game):
        if direction not in self.blocks:
            return False
        block_description, preconditions = self.blocks[direction]
        if all(cond(game) for cond in preconditions):
            return False
        print_game_output(block_description)
        return True

    # ----- Items in location (stacked) -----
    def add_item(self, item, count: int = 1):
        """Add `count` copies of an item into this location."""
        name = item.name
        if name not in self.items:
            self.items[name] = item
        self.item_counts[name] += max(1, int(count))

    def remove_item(self, item_name, count: int = 1):
        """Remove up to `count` of an item; returns a canonical Item or None."""
        item_name = item_name.lower()
        if self.item_counts[item_name] <= 0:
            return None
        item = self.items.get(item_name)
        if item is None:
            return None
        self.item_counts[item_name] -= max(1, int(count))
        if self.item_counts[item_name] <= 0:
            self.item_counts[item_name] = 0
            self.items.pop(item_name, None)
        return item

    # ----- Characters in location -----
    def add_character(self, char_id, count: int = 1):
        self.characters[char_id] += max(1, int(count))

    def get_characters_summary(self):
        return dict(self.characters)


class Item:
    """Items are objects that can be picked up, examined, and used."""

    def __init__(self, item_id, name, description, properties=None):
        self.id = item_id
        self.name = name
        self.description = description
        self.properties = defaultdict(bool)
        if properties:
            for k, v in properties.items():
                self.properties[k] = v

    def set_property(self, prop, value=True):
        self.properties[prop] = value

    def get_property(self, prop):
        return self.properties[prop]


## Game State Tracking

- health and max_health
- insight level
- weapon_level
- inventory
- curr_location
- interactable_objects


In [ ]:

class BloodborneGame:
    def __init__(self, start_at, all_locations,
                 max_health: int = 100,
                 weapon_level: int = 1,
                 starting_insight: int = 0,
                 max_insight: int = 99):
        # World graph
        self.curr_location = start_at
        self.all_locations = all_locations  # dict[loc_id] -> Location

        # Player-centric state
        self.max_health = max_health
        self.health = max_health
        self.weapon_level = weapon_level

        # Insight tracking
        self.max_insight = max_insight
        self.insight = starting_insight

        # Inventory: name -> Item (canonical) plus stacked counts
        self.inventory = {}
        self.inventory_counts = defaultdict(int)

        # Simple currency tracking (Blood Echoes)
        self.blood_echoes = 0

        # Lamps (teleport points) by location id
        self.unlocked_lamp_locations = set()

        # Locations visited at least once
        self.visited_locations = set()

        # Per-location character HP: loc_id -> {char_id: [hp1, hp2, ...]}
        self.character_state = {}

        # Bosses / special enemies made vulnerable via stun items
        self.boss_vulnerable = set()

        # For optional logging of snapshots
        self.state_history = []

        # Initialize character HP pools from metadata if available
        self._initialize_character_state()

        # Treat the starting location as a first-time visit and grant Insight
        self._handle_first_time_location_entry(self.curr_location)

    # ---------- Character HP initialization ----------
    def _initialize_character_state(self):
        """Initialize per-location character health pools.

        Uses CHARACTER_META if available; otherwise defaults each enemy to 50 HP.
        """
        meta = globals().get("CHARACTER_META", {})
        self.character_state = {}
        for loc_id, loc in self.all_locations.items():
            per_loc = {}
            for char_id, count in loc.characters.items():
                base_hp = 50
                if char_id in meta:
                    base_hp = meta[char_id].get("health", 50)
                per_loc[char_id] = [base_hp for _ in range(count)]
            self.character_state[loc_id] = per_loc

    # ---------- Location entry helper ----------
    def _handle_first_time_location_entry(self, location):
        """Mark a location as entered and, if it's the first time ever,
        grant +1 Insight and record the visit.
        """
        loc_id = getattr(location, "id", location.name)
        first_time = loc_id not in self.visited_locations

        self.visited_locations.add(loc_id)
        location.has_been_visited = True

        if first_time:
            print_game_output(f"You glimpse new horrors in {location.name}.")
            self.gain_insight(1)
        else:
            self.log_state()

    # ---------- Movement and description ----------
    def describe(self):
        print_game_output(f"You are at: {self.curr_location.name}")
        print_game_output(self.curr_location.description)
        self.describe_exits()
        self.describe_items_here()
        self.describe_characters_here()
        self.describe_status()

    def describe_exits(self):
        exits = [d.capitalize() for d in self.curr_location.connections.keys()]
        if exits:
            print_game_output("Exits: " + ", ".join(exits))
        else:
            print_game_output("There are no obvious exits.")

    def describe_items_here(self):
        if not self.curr_location.items:
            print_game_output("You see nothing of note here.")
            return
        print_game_output("You see:")
        for name, item in self.curr_location.items.items():
            count = self.curr_location.item_counts[name]
            if count > 1:
                plural_name = name if name.endswith("s") else name + "s"
                print_game_output(f"  - {plural_name} (x{count}): {item.description}")
            else:
                print_game_output(f"  - {name}: {item.description}")

    def describe_characters_here(self):
        chars = self.curr_location.get_characters_summary()
        if not chars:
            return
        print_game_output("You notice:")
        for char_id, count in chars.items():
            label = f"{char_id} (x{count})" if count > 1 else char_id
            print_game_output(f"  - {label}")

    def describe_inventory(self):
        if not self.inventory:
            print_game_output("You are carrying nothing.")
            return
        print_game_output("You are carrying:")
        for name, item in self.inventory.items():
            count = self.inventory_counts[name]
            if count > 1:
                plural_name = name if name.endswith("s") else name + "s"
                print_game_output(f"  - {plural_name} (x{count})")
            else:
                print_game_output(f"  - {name}")

    def describe_status(self):
        print_game_output(
            f"Health: {self.health}/{self.max_health}  |  "
            f"Weapon +{self.weapon_level}  |  "
            f"Insight: {self.insight}/{self.max_insight}  "
            f"(Area Insight: {self.curr_location.insight_level})"
        )
        print_game_output(
            f"Blood Echoes: {self.blood_echoes}  |  Lamps unlocked: {self.lamps_unlocked_count}"
        )

    @property
    def lamps_unlocked_count(self):
        return len(self.unlocked_lamp_locations)

    def go(self, direction):
        direction = direction.lower()
        if direction not in self.curr_location.connections:
            print_game_output("You can't go that way.")
            return
        if self.curr_location.is_blocked(direction, self):
            return

        new_loc = self.curr_location.connections[direction]
        travel_desc = self.curr_location.travel_descriptions.get(direction, "")
        if travel_desc:
            print_game_output(travel_desc)
        self.curr_location = new_loc

        # Handle first-time visit logic and then describe area
        self._handle_first_time_location_entry(self.curr_location)
        self.describe()

    # ---------- Inventory management ----------
    def is_in_inventory(self, item_name):
        item_name = item_name.lower()
        return self.inventory_counts[item_name] > 0

    def add_to_inventory(self, item, count: int = 1):
        name = item.name
        if name not in self.inventory:
            self.inventory[name] = item
        self.inventory_counts[name] += max(1, int(count))
        qty = self.inventory_counts[name]
        if qty > 1:
            plural = name if name.endswith("s") else name + "s"
            print_game_output(f"You now have {plural} (x{qty}).")
        else:
            print_game_output(f"You take the {name}.")
        self.log_state()

    def consume_inventory_item(self, item_name, amount: int = 1):
        item_name = item_name.lower()
        if self.inventory_counts[item_name] <= 0:
            return False
        self.inventory_counts[item_name] -= max(1, int(amount))
        if self.inventory_counts[item_name] <= 0:
            self.inventory_counts[item_name] = 0
            self.inventory.pop(item_name, None)
        self.log_state()
        return True

    def remove_from_inventory(self, item_name, amount: int = 1):
        """Drop `amount` of an item into the current location."""
        item_name = item_name.lower()
        if self.inventory_counts[item_name] <= 0:
            print_game_output("You don't have that.")
            return None

        item = self.inventory.get(item_name)
        if item is None:
            print_game_output("You don't have that.")
            return None

        self.inventory_counts[item_name] -= max(1, int(amount))
        dropped_count = 1
        if self.inventory_counts[item_name] <= 0:
            self.inventory_counts[item_name] = 0
            self.inventory.pop(item_name, None)
        self.curr_location.add_item(item, count=dropped_count)
        print_game_output(f"You drop the {item.name}.")
        self.log_state()
        return item

    def take(self, item_name, count: int = 1):
        item_name = item_name.lower()
        if item_name not in self.curr_location.items:
            print_game_output("There is no such item here.")
            return
        item = self.curr_location.remove_item(item_name, count=count)
        if item is None:
            print_game_output("There is no such item here.")
            return
        self.add_to_inventory(item, count=count)

    # ---------- Combat, Health, Weapon Upgrades ----------
    def take_damage(self, amount):
        self.health = max(0, self.health - amount)
        print_game_output(f"You take {amount} damage! Health is now {self.health}.")
        self.log_state()

    def heal(self, amount):
        old = self.health
        self.health = min(self.max_health, self.health + amount)
        restored = self.health - old
        print_game_output(f"You recover {restored} health. Health is now {self.health}.")
        self.log_state()

    def upgrade_weapon(self, levels: int = 1):
        self.weapon_level += levels
        print_game_output(f"Your weapon has been strengthened to +{self.weapon_level}.")
        self.log_state()

    # ---------- Blood Echoes ----------
    def gain_blood_echoes(self, amount: int):
        if amount <= 0:
            return
        self.blood_echoes += amount
        print_game_output(f"You gain {amount} Blood Echoes. Total: {self.blood_echoes}.")
        self.log_state()

    def spend_blood_echoes(self, amount: int) -> bool:
        if amount <= 0:
            return True
        if self.blood_echoes < amount:
            print_game_output("You lack sufficient Blood Echoes.")
            return False
        self.blood_echoes -= amount
        print_game_output(f"You spend {amount} Blood Echoes. Remaining: {self.blood_echoes}.")
        self.log_state()
        return True

    # ---------- Insight management ----------
    def gain_insight(self, amount: int = 1):
        old = self.insight
        self.insight = min(self.max_insight, self.insight + amount)
        gained = self.insight - old
        if gained > 0:
            print_game_output(f"You gain {gained} Insight (now {self.insight}/{self.max_insight}).")
        self.log_state()

    def lose_insight(self, amount: int = 1):
        old = self.insight
        self.insight = max(0, self.insight - amount)
        lost = old - self.insight
        if lost > 0:
            print_game_output(f"You lose {lost} Insight (now {self.insight}/{self.max_insight}).")
        self.log_state()

    # ---------- Lamps ----------
    def unlock_lamp_here(self):
        loc_id = getattr(self.curr_location, "id", self.curr_location.name)
        if loc_id in self.unlocked_lamp_locations:
            print_game_output("The lamp here is already lit.")
            return
        self.unlocked_lamp_locations.add(loc_id)
        print_game_output(
            f"You light the lamp at {self.curr_location.name}. Lamps unlocked: {self.lamps_unlocked_count}."
        )
        self.log_state()

    # ---------- State snapshot ----------
    def get_state(self):
        """Return a dictionary capturing the current game state."""
        inventory_display = []
        for name in self.inventory.keys():
            count = self.inventory_counts[name]
            if count > 1:
                inventory_display.append(f"{name} (x{count})")
            else:
                inventory_display.append(name)

        loc_item_summary = {}
        for name, itm in self.curr_location.items.items():
            loc_item_summary[name] = self.curr_location.item_counts[name]

        loc_char_summary = self.curr_location.get_characters_summary()

        return {
            "location_id": getattr(self.curr_location, "id", self.curr_location.name),
            "location": self.curr_location.name,
            "health": self.health,
            "max_health": self.max_health,
            "weapon_level": self.weapon_level,
            "inventory": inventory_display,
            "location_items": loc_item_summary,
            "location_characters": loc_char_summary,
            "insight": self.insight,
            "max_insight": self.max_insight,
            "location_insight": self.curr_location.insight_level,
            "blood_echoes": self.blood_echoes,
            "lamps_unlocked": self.lamps_unlocked_count,
            "visited_locations": list(self.visited_locations),
        }

    def log_state(self):
        """Append a snapshot of the current state to `state_history`."""
        self.state_history.append(self.get_state())


## Define Locations for World Map (TEMP)

- **Hunter's Dream** – a safe hub, with a **workshop altar** for upgrading your weapon
- **Central Yharnam Gate** – beasts lurk; you can pick up **blood vials**
- **Cathedral Ward** – tougher enemies but better rewards

In [ ]:

def _load_item_definitions_from_data_zip(zip_path: str = DATA_ZIP_PATH):
    """Load all item definitions from Data.zip and build Item objects.

    Returns:
        dict[item_id] -> Item
    """
    item_defs = {}
    if not os.path.isfile(zip_path):
        return item_defs

    with zipfile.ZipFile(zip_path, "r") as z:
        for name in z.namelist():
            if not (name.startswith("Data/Items/") and name.endswith(".json")):
                continue
            with z.open(name) as f:
                data = json.load(f)
            item_id = data.get("id")
            item_name = data.get("name", item_id).lower()
            desc = data.get("description", "")
            # Everything except id/name/description is treated as a property
            props = {}
            for k, v in data.items():
                if k in ("id", "name", "description"):
                    continue
                props[k] = v
            item = Item(item_id=item_id, name=item_name, description=desc, properties=props)
            item_defs[item_id] = item
    return item_defs


def build_world():
    """Build the world graph from the JSON files inside Data.zip.

    Locations, items, and characters are all pulled from the dataset and kept
    in memory by the game controller.
    """
    zip_path = DATA_ZIP_PATH
    if not os.path.isfile(zip_path):
        raise FileNotFoundError(f"Could not find dataset zip at {zip_path}")

    item_defs = _load_item_definitions_from_data_zip(zip_path)

    locations_by_id = {}
    raw_location_data = {}

    with zipfile.ZipFile(zip_path, "r") as z:
        for name in z.namelist():
            if not (name.startswith("Data/Locations/") and name.endswith(".json")):
                continue
            with z.open(name) as f:
                data = json.load(f)

            loc_id = data.get("id")
            loc_name = data.get("name", loc_id)
            desc = data.get("description", "")
            insight_level = data.get("insight_level", 0)

            loc = Location(loc_name, desc, insight_level=insight_level, loc_id=loc_id)
            locations_by_id[loc_id] = loc
            raw_location_data[loc_id] = data

    # Hook up previous/next as a simple linear chain (previous -> west, next -> east)
    for loc_id, data in raw_location_data.items():
        loc = locations_by_id[loc_id]
        prev_id = data.get("previous")
        next_id = data.get("next")

        if prev_id and prev_id in locations_by_id:
            loc.add_connection("west", locations_by_id[prev_id],
                               travel_description="You push back toward where you came from.")
        if next_id and next_id in locations_by_id:
            loc.add_connection("east", locations_by_id[next_id],
                               travel_description="You push forward into the unknown.")

        # Characters present in this location
        for ch in data.get("characters", []):
            cid = ch.get("id")
            count = ch.get("count", 1)
            if cid:
                loc.add_character(cid, count=count)

        # Items present in this location
        for it in data.get("items", []):
            item_id = it.get("item_id")
            count = it.get("count", 1)
            if item_id and item_id in item_defs:
                item_obj = item_defs[item_id]
                loc.add_item(item_obj, count=count)

    # Choose a starting location:
    #  - Prefer one with no 'previous', else just pick any deterministic one.
    start_location = None
    for loc_id, data in raw_location_data.items():
        if not data.get("previous"):
            start_location = locations_by_id[loc_id]
            break
    if start_location is None and locations_by_id:
        # Fallback: first in sorted order
        first_id = sorted(locations_by_id.keys())[0]
        start_location = locations_by_id[first_id]

    if start_location is None:
        raise RuntimeError("No locations were loaded from the dataset.")

    return start_location, locations_by_id


## Initialize the Game

In [ ]:

# === Action Set Implementations ===

def _require_game():
    """Helper to fetch the global `game` instance used by the controller."""
    if 'game' not in globals():
        print_game_output("No active game. Initialize the game first.")
        return None
    return globals()['game']


def _normalize_name(name):
    if not isinstance(name, str):
        return ""
    return name.strip().lower()


def _find_character_ids_in_location(game, target_name):
    """Return a list of character ids in the current location that match `target_name`."""
    loc = game.curr_location
    chars_here = loc.get_characters_summary()
    meta = globals().get("CHARACTER_META", {})

    if target_name is None:
        return list(chars_here.keys())

    t = _normalize_name(target_name)
    matches = []
    for char_id in chars_here.keys():
        if t in _normalize_name(char_id):
            matches.append(char_id)
            continue
        if char_id in meta:
            if t in _normalize_name(meta[char_id].get("name", "")):
                matches.append(char_id)
    return matches


def _ensure_character_state_for(game, char_id):
    """Ensure character_state entries exist for current location + char_id."""
    meta = globals().get("CHARACTER_META", {})
    loc_id = getattr(game.curr_location, "id", game.curr_location.name)
    loc_state = game.character_state.setdefault(loc_id, {})
    if char_id not in loc_state:
        base_hp = 50
        if char_id in meta:
            base_hp = meta[char_id].get("health", 50)
        count = game.curr_location.characters.get(char_id, 0)
        loc_state[char_id] = [base_hp for _ in range(count)]
    return loc_state[char_id]


def Attack(object, instrument):
    ACTION_NAME = "attack"
    ACTION_DESCRIPTION = "Attack someone with a weapon"
    ACTION_ALIASES = ["hit", "stab", "shoot", "strike", "slash"]

    game = _require_game()
    if game is None:
        return

    if game.health <= 0:
        print_game_output("You are in no condition to fight. (You are dead.)")
        return

    if instrument is None:
        print_game_output("You must specify a weapon to attack with.")
        return

    weapon_name = _normalize_name(instrument)
    if not game.is_in_inventory(weapon_name):
        print_game_output(f"You are not holding a {instrument}.")
        return

    # Determine enemies in the room
    loc = game.curr_location
    chars_here = loc.get_characters_summary()
    if not chars_here:
        print_game_output("There is nothing here to attack.")
        return

    # Select targets
    if object is None or _normalize_name(object) in ("room", "all"):
        target_ids = list(chars_here.keys())
    else:
        target_ids = _find_character_ids_in_location(game, object)
        if not target_ids:
            print_game_output(f"There is no '{object}' here to attack.")
            return

    item_meta = globals().get("ITEM_META", {})
    weapon_meta = item_meta.get(weapon_name, {})
    base_damage = weapon_meta.get("damage", 30)
    total_damage = int(base_damage * (1 + 0.20 * (game.weapon_level - 1)))

    # Gather individual enemies (char_id, index) with HP > 0
    loc_id = getattr(loc, "id", loc.name)
    individual_targets = []
    for cid in target_ids:
        hp_list = _ensure_character_state_for(game, cid)
        for idx, hp in enumerate(hp_list):
            if hp > 0:
                individual_targets.append((cid, idx))

    if not individual_targets:
        print_game_output("All foes here have already fallen.")
        return

    per_target_damage = max(1, total_damage // len(individual_targets))

    meta = globals().get("CHARACTER_META", {})
    drop_log = []

    # Apply damage
    for cid, idx in individual_targets:
        hp_list = game.character_state[loc_id][cid]
        old_hp = hp_list[idx]
        new_hp = max(0, old_hp - per_target_damage)
        hp_list[idx] = new_hp
        print_game_output(f"You hit {cid} #{idx+1} for {per_target_damage} damage (HP: {new_hp}).")

    # Recompute counts and handle deaths
    for cid in target_ids:
        hp_list = game.character_state[loc_id][cid]
        alive = [hp for hp in hp_list if hp > 0]
        dead_count = len(hp_list) - len(alive)
        if dead_count > 0:
            print_game_output(f"{dead_count} {cid}(s) are slain!")
            hp_list[:] = alive  # keep only survivors
            loc.characters[cid] = len(alive)
            if cid in meta:
                for drop in meta[cid].get("drop_items", []):
                    name = drop.get("item_name")
                    qty = int(drop.get("quantity", 1))
                    if not name:
                        continue
                    if _normalize_name(name) == "blood echoes":
                        game.gain_blood_echoes(qty)
                    else:
                        # Create a simple Item for the drop
                        item_name_norm = _normalize_name(name)
                        item_info = item_meta.get(item_name_norm, {})
                        desc = item_info.get("description", f"An item dropped by {cid}.")
                        props = {}
                        for k, v in item_info.items():
                            if k in ("id", "name", "description"):
                                continue
                            props[k] = v
                        dropped = Item(
                            item_id=item_info.get("id", item_name_norm),
                            name=item_name_norm,
                            description=desc,
                            properties=props
                        )
                        loc.add_item(dropped, count=qty)
                        drop_log.append(f"{name} (x{qty})")

    if drop_log:
        print_game_output("Dropped items:")
        for line in drop_log:
            print_game_output(f"  - {line}")

    # Simple counterattack: surviving enemies deal damage
    incoming = 0
    for cid, hp_list in game.character_state.get(loc_id, {}).items():
        if cid not in meta:
            continue
        dmg_per = meta[cid].get("damage", 10)
        alive_units = sum(1 for hp in hp_list if hp > 0)
        incoming += dmg_per * alive_units

    if incoming > 0:
        # Soften it a bit
        incoming = max(1, incoming // 2)
        game.take_damage(incoming)

    # Potential insight on boss discovery / defeat (simple heuristic)
    for cid in target_ids:
        info = meta.get(cid, {})
        if info.get("stun_item"):
            # Treat as a boss
            if cid in game.character_state.get(loc_id, {}) and all(hp <= 0 for hp in game.character_state[loc_id][cid]):
                # boss dead
                game.gain_insight(1)

    # Show updated stats
    Player_Stats()


def Talk(obj):
    ACTION_NAME = "talk"
    ACTION_DESCRIPTION = "Talk to an NPC"
    ACTION_ALIASES = ["speak", "chat", "communicate"]

    game = _require_game()
    if game is None:
        return

    if game.health <= 0:
        print_game_output("You mumble something, but only the void answers. (You are dead.)")
        return

    loc = game.curr_location
    chars_here = loc.get_characters_summary()
    if not chars_here:
        print_game_output("No one here seems interested in conversation.")
        return

    if obj is None:
        print_game_output("You can talk to:")
        for cid, count in chars_here.items():
            label = f"{cid} (x{count})" if count > 1 else cid
            print_game_output(f"  - {label}")
        return

    meta = globals().get("CHARACTER_META", {})
    matches = _find_character_ids_in_location(game, obj)
    if not matches:
        print_game_output(f"There is no '{obj}' here to talk to.")
        return

    cid = matches[0]
    info = meta.get(cid, {})
    name = info.get("name", cid)
    friendly = info.get("friendly", False)

    if not friendly:
        print_game_output(f"{name} does not seem interested in small talk...")
        return

    # Placeholder dialogue system
    print_game_output(f"{name} regards you carefully.")
    print_game_output(f"\"We are trapped in this nightmare together, hunter. (Insight: {game.insight})\"")


def Use(object):
    ACTION_NAME = "use"
    ACTION_DESCRIPTION = "Use an item from inventory"
    ACTION_ALIASES = ["consume", "activate"]

    game = _require_game()
    if game is None:
        return

    if object is None:
        print_game_output("Use what?")
        return

    item_name = _normalize_name(object)
    if not game.is_in_inventory(item_name):
        print_game_output(f"You don't seem to have any {object}.")
        return

    meta_items = globals().get("ITEM_META", {})
    info = meta_items.get(item_name, {})
    category = info.get("category", "")

    # Blood vial -> heal
    if item_name == "blood vial":
        if game.consume_inventory_item(item_name, 1):
            game.heal(35)
            Player_Stats()
        return

    # Bloodstone Shard -> weapon upgrade
    if item_name == "bloodstone shard" or category == "upgrade_item":
        Upgrade_Weapon(object)
        return

    # Currency Blood Echoes -> route to Increase_Health
    if item_name == "blood echoes" or category == "currency":
        Increase_Health()
        return

    # Lamp-like items: show lit lamps for teleport
    if "lamp" in item_name or category == "lamp":
        if not game.unlocked_lamp_locations:
            print_game_output("No lamps have been lit yet.")
        else:
            print_game_output("Lamps lit at:")
            locs = list(game.unlocked_lamp_locations)
            for idx, loc_id in enumerate(locs, start=1):
                loc_obj = game.all_locations.get(loc_id)
                lname = loc_obj.name if loc_obj else loc_id
                print_game_output(f"  {idx}. {lname}")
        return

    # Stun items: set boss vulnerable
    if category == "stun_item":
        target_name = info.get("stun_target")
        char_meta = globals().get("CHARACTER_META", {})
        boss_id = None
        if target_name:
            tn = _normalize_name(target_name)
            for cid, cinfo in char_meta.items():
                if tn in _normalize_name(cinfo.get("name", "")):
                    boss_id = cid
                    break
        if boss_id:
            game.boss_vulnerable.add(boss_id)
            print_game_output(f"You use {object}. {char_meta[boss_id].get('name', boss_id)} seems shaken!")
        else:
            print_game_output(f"You use {object}, but nothing obvious happens.")
        game.consume_inventory_item(item_name, 1)
        return

    # Generic item use
    print_game_output(f"You use the {object}, but its effects are subtle or not yet implemented.")
    game.consume_inventory_item(item_name, 1)


def Upgrade_Weapon(object):
    ACTION_NAME = "upgrade_weapon"
    ACTION_DESCRIPTION = "Upgrade a weapon using Blood Shards"
    ACTION_ALIASES = ["upgrade", "enhance", "fortify"]

    game = _require_game()
    if game is None:
        return

    loc_name = _normalize_name(game.curr_location.name)
    if "workshop" not in loc_name and "dream" not in loc_name:
        print_game_output("You can only upgrade your weapon at a workshop in the Hunter's Dream.")
        return

    shard_name = "bloodstone shard"
    if not game.is_in_inventory(shard_name):
        print_game_output("You have no Bloodstone Shards to perform an upgrade.")
        return

    if game.consume_inventory_item(shard_name, 1):
        game.upgrade_weapon(1)
        Inventory_Stats()


def Heal():
    ACTION_NAME = "heal"
    ACTION_DESCRIPTION = "Heal yourself"
    ACTION_ALIASES = ["restore", "cure"]

    game = _require_game()
    if game is None:
        return

    if game.health <= 0:
        print_game_output("You cannot heal. You are already dead.")
        return

    item_name = "blood vial"
    if not game.is_in_inventory(item_name):
        print_game_output("You fumble for a blood vial, but have none left.")
        return

    if game.consume_inventory_item(item_name, 1):
        game.heal(35)
        Player_Stats()


def Increase_Health():
    ACTION_NAME = "increase_health"
    ACTION_DESCRIPTION = "Increase player's maximum health"
    ACTION_ALIASES = ["boost_health", "enhance_vitality"]

    game = _require_game()
    if game is None:
        return

    loc_name = _normalize_name(game.curr_location.name)
    if "lawn" not in loc_name and "dream" not in loc_name:
        print_game_output("You feel no power here to reshape your flesh.")
        return

    # Require a friendly Doll-like NPC if present in dataset
    meta = globals().get("CHARACTER_META", {})
    doll_present = False
    for cid in game.curr_location.characters.keys():
        info = meta.get(cid, {})
        if "doll" in _normalize_name(info.get("name", "")) and info.get("friendly", False):
            doll_present = True
            break
    if not doll_present and meta:
        print_game_output("No one is here to channel your Blood Echoes.")
        return

    cost = 100
    if game.blood_echoes < cost:
        print_game_output(f"You need at least {cost} Blood Echoes to strengthen yourself.")
        return

    if not game.spend_blood_echoes(cost):
        return

    from math import ceil
    delta = max(1, ceil(game.max_health * 0.05))
    game.max_health += delta
    print_game_output(f"Your vitality surges. Max health increased by {delta} (now {game.max_health}).")
    Player_Stats()


def Travel(object):
    ACTION_NAME = "travel"
    ACTION_DESCRIPTION = "Travel between connected locations"
    ACTION_ALIASES = ["go", "walk"]

    game = _require_game()
    if game is None:
        return

    if object is None:
        print_game_output("Travel where? (north/south/east/west)")
        return

    direction = _normalize_name(object)
    if direction in ("left", "west"):
        direction = "west"
    elif direction in ("right", "east"):
        direction = "east"
    elif direction in ("forward", "north"):
        direction = "north"
    elif direction in ("back", "backward", "south"):
        direction = "south"

    game.go(direction)


def Teleport(object):
    ACTION_NAME = "teleport"
    ACTION_DESCRIPTION = "Teleport to a previously visited location using a lamp"
    ACTION_ALIASES = ["warp", "move"]

    game = _require_game()
    if game is None:
        return

    if not game.unlocked_lamp_locations:
        print_game_output("You have not lit any lamps yet.")
        return

    lamp_loc_ids = sorted(list(game.unlocked_lamp_locations))
    if object is None:
        print_game_output("Available lamp destinations:")
        for idx, loc_id in enumerate(lamp_loc_ids, start=1):
            loc = game.all_locations.get(loc_id)
            name = loc.name if loc else loc_id
            print_game_output(f"  {idx}. {name}")
        return

    try:
        idx = int(object)
    except (TypeError, ValueError):
        print_game_output("Teleport expects the number of a lamp destination.")
        return

    if idx < 1 or idx > len(lamp_loc_ids):
        print_game_output("That lamp choice is invalid.")
        return

    dest_id = lamp_loc_ids[idx - 1]
    if dest_id not in game.all_locations:
        print_game_output("That destination no longer exists.")
        return

    dest = game.all_locations[dest_id]
    if dest is game.curr_location:
        print_game_output("You are already at that lamp.")
        return

    game.curr_location = dest
    game._handle_first_time_location_entry(dest)
    game.describe()


def Get(object):
    ACTION_NAME = "get"
    ACTION_DESCRIPTION = "Pick up an item"
    ACTION_ALIASES = ["take", "grab", "pickup", "collect"]

    game = _require_game()
    if game is None:
        return

    if object is None:
        print_game_output("Take what?")
        return

    item_name = _normalize_name(object)
    if item_name not in game.curr_location.items:
        print_game_output(f"There is no {object} here.")
        return

    meta_items = globals().get("ITEM_META", {})
    info = meta_items.get(item_name, {})
    if info is not None and info.get("gettable") is False:
        print_game_output(f"You cannot take the {object}.")
        return

    # Determine how many to take: for now, 1
    game.take(item_name, count=1)
    Inventory_Stats()


def Inspect(object):
    ACTION_NAME = "inspect"
    ACTION_DESCRIPTION = "Examine an item or character for details"
    ACTION_ALIASES = ["examine", "look", "check", "read"]

    game = _require_game()
    if game is None:
        return

    if object is None:
        print_game_output("Inspect what?")
        return

    target = _normalize_name(object)
    meta_items = globals().get("ITEM_META", {})
    meta_chars = globals().get("CHARACTER_META", {})

    # 1) Check inventory items
    if target in game.inventory:
        info = meta_items.get(target, {})
        print_game_output(f"{object}: {info.get('description', game.inventory[target].description)}")
        print_game_output(f"Category: {info.get('category', 'unknown')}")
        if "damage" in info:
            print_game_output(f"Damage: {info['damage']}")
        if "healing" in info:
            print_game_output(f"Healing: {info['healing']}")
        print_game_output(f"Gettable: {info.get('gettable', True)}")
        return

    # 2) Check items in current location
    if target in game.curr_location.items:
        item = game.curr_location.items[target]
        info = meta_items.get(target, {})
        print_game_output(f"{object}: {info.get('description', item.description)}")
        print_game_output(f"Category: {info.get('category', 'unknown')}")
        if "damage" in info:
            print_game_output(f"Damage: {info['damage']}")
        if "healing" in info:
            print_game_output(f"Healing: {info['healing']}")
        print_game_output(f"Gettable: {info.get('gettable', True)}")
        return

    # 3) Check characters in current location
    for cid in game.curr_location.characters.keys():
        info = meta_chars.get(cid, {})
        name = info.get("name", cid)
        if target in _normalize_name(cid) or target in _normalize_name(name):
            friendly = info.get("friendly", False)
            status = "friendly" if friendly else "hostile"
            hp = info.get("health", 0)
            dmg = info.get("damage", 0)
            vuln = "Yes" if info.get("stun_item") else "No"
            print_game_output(f"{name} ({cid}) – {status}")
            if info.get("description"):
                print_game_output(info["description"])
            print_game_output(f"Health: {hp}, Damage: {dmg}, Vulnerable to stun item: {vuln}")
            return

    print_game_output(f"You see no {object} here to inspect.")


def Inventory_Stats():
    ACTION_NAME = "inventory_stats"
    ACTION_DESCRIPTION = "Display player's inventory statistics"
    ACTION_ALIASES = ["inventory stats", "show inventory", "display inventory", "inventory"]

    game = _require_game()
    if game is None:
        return

    if not game.inventory:
        print_game_output("You are not carrying anything.")
        return

    meta_items = globals().get("ITEM_META", {})
    print_game_output("Inventory:")
    for name, item in game.inventory.items():
        count = game.inventory_counts[name]
        info = meta_items.get(name, {})
        cat = info.get("category", "unknown")
        if cat == "weapon":
            print_game_output(f"  - {name} (x{count}) – weapon +{game.weapon_level}")
        else:
            print_game_output(f"  - {name} (x{count}) – {cat}")


def Describe_Room():
    ACTION_NAME = "describe_room"
    ACTION_DESCRIPTION = "Describe the current room/location"
    ACTION_ALIASES = ["look around", "examine room", "room description"]

    game = _require_game()
    if game is None:
        return

    game.describe()


def Player_Stats():
    ACTION_NAME = "player_stats"
    ACTION_DESCRIPTION = "Display player's current statistics"
    ACTION_ALIASES = ["show stats", "display stats", "stats"]

    game = _require_game()
    if game is None:
        return

    print_game_output(f"Location: {game.curr_location.name} (id: {getattr(game.curr_location, 'id', game.curr_location.name)})")
    game.describe_status()
    # "Blood shards" = Bloodstone Shards in inventory
    shards = game.inventory_counts.get("bloodstone shard", 0)
    print_game_output(f"Blood Shards: {shards}")


In [12]:
start_location, all_locations = build_world()
game = BloodborneGame(
    start_location,
    all_locations,
    max_health=100,
    weapon_level=1,
    starting_insight=0,   # start “blind”
    max_insight=99
)

game.log_state()
game.describe()
print("\nInitial game state snapshot:")
print(game.get_state())


You are at: Hunter's Dream
A pale, moonlit workshop floating above a forgotten city. A comforting fire burns nearby.
Exits: North
You see:
  - workshop altar: An intricate altar where you can channel blood echoes to strengthen your weapon.
Health: 100/100  |  Weapon +1  |  Insight: 0/99  (Area Insight: 0)

Initial game state snapshot:
{'location': "Hunter's Dream", 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': [], 'interactable_objects': ['workshop altar'], 'insight': 0, 'max_insight': 99, 'location_insight': 0}


## Example: State Tracking 

In [7]:
# Example scripted playthrough to show state tracking
print("--- Move north to Central Yharnam ---")
game.go('north')
print("Current state:")
print(game.get_state())

print("\n--- Take the blood vial ---")
game.take('blood vial')
print("Current state:")
print(game.get_state())

print("\n--- Take damage from a beast ---")
game.take_damage(45)
print("Current state:")
print(game.get_state())

print("\n--- Use a blood vial to heal ---")
use_blood_vial(game)
print("Current state:")
print(game.get_state())

print("\n--- Return to Hunter's Dream and upgrade weapon ---")
game.go('south')  # back to Hunter's Dream
use_workshop_altar(game)
print("Current state:")
print(game.get_state())

print("\n--- Move to Cathedral Ward ---")
game.go('north')      # to Central Yharnam
game.go('east')       # to Cathedral Ward
print("Current state:")
print(game.get_state())

print("\nState history (first 3 snapshots):")
for i, s in enumerate(game.state_history[:3]):
    print(i, s)


--- Move north to Central Yharnam ---
You step through the headstone and awaken in Yharnam...
You are at: Central Yharnam Gate
Cobblestone streets slick with blood. Lanterns flicker as distant howls echo through the fog.
Exits: South, East
You see:
  - blood vial: A small glass vial of thick, red blood. Restores a portion of health when used.
Health: 100/100  |  Weapon +1
Current state:
{'location': 'Central Yharnam Gate', 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': [], 'interactable_objects': ['blood vial']}

--- Take the blood vial ---
You take the blood vial.
Current state:
{'location': 'Central Yharnam Gate', 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': ['blood vial'], 'interactable_objects': []}

--- Take damage from a beast ---
You take 45 damage! Health is now 55.
Current state:
{'location': 'Central Yharnam Gate', 'health': 55, 'max_health': 100, 'weapon_level': 1, 'inventory': ['blood vial'], 'interactable_objects': []}

--- Use a blo